# Feature Importance Analysis - Kaggle Notebook

## 🎯 Objective
Analyze which behavioral features are most predictive of mental health outcomes (stress, mood, energy, focus, etc.)

## 📋 Prerequisites
**Before running this notebook:**
1. Upload your dataset to Kaggle:
   - Go to "Add Data" → "Upload" → Select `daily_logs.csv` and `weekly_summaries.csv`
   - OR use an existing Kaggle dataset
2. Update the `KAGGLE_INPUT_PATH` variable in the next cell to match your dataset location

## 🔄 What This Notebook Does
1. ✅ Fixes file path issues automatically
2. ✅ Creates burnout labels from raw data
3. ✅ Runs 3 feature importance methods (Random Forest, Permutation, Correlation)
4. ✅ Generates visualizations for all mental health targets

In [ ]:
# ============================================================================
# SETUP - UPDATE THIS PATH TO YOUR KAGGLE DATASET!
# ============================================================================

# 🔧 CHANGE THIS to match your Kaggle dataset path
# Example: '/kaggle/input/mental-health-daily-logs/'
# Example: '/kaggle/input/burnout-prediction-dataset/'
KAGGLE_INPUT_PATH = '/kaggle/input/your-dataset-name-here/'

# Check if path exists
import os
from pathlib import Path

print("=" * 80)
print("CHECKING KAGGLE ENVIRONMENT")
print("=" * 80)

if Path(KAGGLE_INPUT_PATH).exists():
    print(f"✓ Dataset found: {KAGGLE_INPUT_PATH}")
    print(f"✓ Available files:")
    for f in os.listdir(KAGGLE_INPUT_PATH):
        print(f"  - {f}")
else:
    print(f"❌ Dataset not found: {KAGGLE_INPUT_PATH}")
    print(f"\n📁 Available datasets in /kaggle/input/:")
    if Path('/kaggle/input').exists():
        for d in os.listdir('/kaggle/input/'):
            print(f"  - {d}")
    print("\n⚠️  UPDATE KAGGLE_INPUT_PATH variable above with the correct path!")

In [ ]:
# ============================================================================
# STEP 1: Clone Repository & Setup
# ============================================================================

!git clone https://github.com/danpinocontrollino/FDS-Project.git
%cd FDS-Project

# Create directory structure
!mkdir -p data/raw data/processed

# Copy raw data from Kaggle input to expected location
import shutil

# Update these filenames if your files have different names
daily_logs_file = 'daily_logs.csv'  # or 'daily_all.csv' or 'daily.csv'
weekly_summary_file = 'weekly_summaries.csv'  # or 'weekly.csv'

# Try to copy files
try:
    src = f'{KAGGLE_INPUT_PATH}/{daily_logs_file}'
    dst = 'data/raw/daily_logs.csv'
    shutil.copy(src, dst)
    print(f"✓ Copied {daily_logs_file} → {dst}")
except Exception as e:
    print(f"❌ Error copying {daily_logs_file}: {e}")
    print(f"   Available files: {os.listdir(KAGGLE_INPUT_PATH)}")

try:
    src = f'{KAGGLE_INPUT_PATH}/{weekly_summary_file}'
    dst = 'data/raw/weekly_summaries.csv'
    shutil.copy(src, dst)
    print(f"✓ Copied {weekly_summary_file} → {dst}")
except Exception as e:
    print(f"❌ Error copying {weekly_summary_file}: {e}")

In [ ]:
# ============================================================================
# STEP 2: Create Burnout Labels
# ============================================================================

print("\n" + "=" * 80)
print("CREATING BURNOUT LABELS FROM RAW DATA")
print("=" * 80)

!python scripts/create_burnout_labels.py

# Verify output
if Path('data/processed/daily_with_burnout.parquet').exists():
    print("\n✓ Successfully created daily_with_burnout.parquet")
else:
    print("\n❌ Failed to create processed data files")

In [ ]:
# ============================================================================
# STEP 3: Load & Inspect Data
# ============================================================================

import pandas as pd
import numpy as np

# Load preprocessed data
df = pd.read_parquet('data/processed/daily_with_burnout.parquet')

print("=" * 80)
print("DATA LOADED SUCCESSFULLY")
print("=" * 80)
print(f"✓ Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"✓ Memory: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"\n📊 First few rows:")
display(df.head())

print(f"\n📋 Available columns:")
for col in df.columns:
    print(f"  - {col}")

In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Feature columns (17 behavioral features)
FEATURE_COLS = [
    "sleep_hours", "sleep_quality", "work_hours", "meetings_count",
    "tasks_completed", "emails_received", "commute_minutes",
    "exercise_minutes", "steps_count", "caffeine_mg", "alcohol_units",
    "screen_time_hours", "social_interactions", "outdoor_time_minutes",
    "diet_quality", "work_pressure", "weather_mood_impact",
]

# Target columns - check which are available in your data
TARGET_COLS = [
    "stress_level", "mood_score", "energy_level", "focus_score",
    "perceived_stress_scale", "anxiety_score", "depression_score",
    "job_satisfaction", "burnout_score"
]

# Filter to only available targets
TARGET_COLS = [t for t in TARGET_COLS if t in df.columns]
print(f"✓ Will analyze {len(TARGET_COLS)} targets: {TARGET_COLS}")

FEATURE_NAMES = {
    "sleep_hours": "Sleep Hours", "sleep_quality": "Sleep Quality",
    "work_hours": "Work Hours", "meetings_count": "Meetings",
    "tasks_completed": "Tasks", "emails_received": "Emails",
    "commute_minutes": "Commute", "exercise_minutes": "Exercise",
    "steps_count": "Steps", "caffeine_mg": "Caffeine",
    "alcohol_units": "Alcohol", "screen_time_hours": "Screen Time",
    "social_interactions": "Social", "outdoor_time_minutes": "Outdoor Time",
    "diet_quality": "Diet", "work_pressure": "Work Pressure",
    "weather_mood_impact": "Weather Impact",
}

# Sample for faster analysis
SAMPLE_SIZE = 50000
if len(df) > SAMPLE_SIZE:
    print(f"📉 Sampling {SAMPLE_SIZE:,} rows for faster analysis...")
    df = df.sample(n=SAMPLE_SIZE, random_state=42)

In [ ]:
# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def prepare_data(df, target):
    """Prepare features and target for analysis."""
    print(f"\n{'='*80}")
    print(f"PREPARING DATA FOR: {target.upper()}")
    print(f"{'='*80}")
    
    available_features = [f for f in FEATURE_COLS if f in df.columns]
    print(f"✓ Found {len(available_features)}/{len(FEATURE_COLS)} features")
    
    if target not in df.columns:
        print(f"❌ Target '{target}' not found")
        return None, None, None
    
    df_clean = df[available_features + [target]].copy().dropna(subset=[target])
    
    # Handle categorical features
    if 'work_pressure' in df_clean.columns:
        pressure_map = {'low': 0, 'medium': 1, 'high': 2}
        df_clean['work_pressure'] = df_clean['work_pressure'].map(pressure_map)
    
    # Fill missing
    for col in available_features:
        if df_clean[col].isna().any():
            df_clean[col].fillna(df_clean[col].mean(), inplace=True)
    
    X = df_clean[available_features].values
    y = df_clean[target].values
    
    print(f"✓ Dataset: {len(X):,} samples × {len(available_features)} features")
    print(f"✓ Target: mean={y.mean():.2f}, std={y.std():.2f}")
    
    return X, y, available_features

In [ ]:
def random_forest_importance(X, y, feature_names):
    """Calculate feature importance using Random Forest."""
    print(f"\n{'='*80}")
    print("METHOD 1: RANDOM FOREST FEATURE IMPORTANCE")
    print(f"{'='*80}")
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    train_score = rf.score(X_train, y_train)
    test_score = rf.score(X_test, y_test)
    print(f"✓ R² Score - Train: {train_score:.3f}, Test: {test_score:.3f}")
    
    importances = rf.feature_importances_
    df_importance = pd.DataFrame({
        'Feature': [FEATURE_NAMES.get(f, f) for f in feature_names],
        'Importance': importances,
        'Importance_pct': importances * 100
    }).sort_values('Importance', ascending=False)
    
    print("\n🏆 Top 10 Features:")
    for idx, row in df_importance.head(10).iterrows():
        print(f"  {idx+1}. {row['Feature']}: {row['Importance_pct']:.2f}%")
    
    return df_importance

In [ ]:
def permutation_importance_analysis(X, y, feature_names):
    """Calculate permutation importance."""
    print(f"\n{'='*80}")
    print("METHOD 2: PERMUTATION IMPORTANCE")
    print(f"{'='*80}")
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    perm_imp = permutation_importance(rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
    
    df_importance = pd.DataFrame({
        'Feature': [FEATURE_NAMES.get(f, f) for f in feature_names],
        'Importance': perm_imp.importances_mean,
        'Std': perm_imp.importances_std
    }).sort_values('Importance', ascending=False)
    
    print("\n🏆 Top 10 Features:")
    for idx, row in df_importance.head(10).iterrows():
        print(f"  {idx+1}. {row['Feature']}: {row['Importance']:.4f} ± {row['Std']:.4f}")
    
    return df_importance

In [ ]:
def correlation_analysis(df, target, feature_cols):
    """Calculate correlations with target."""
    print(f"\n{'='*80}")
    print("METHOD 3: CORRELATION ANALYSIS")
    print(f"{'='*80}")
    
    correlations = []
    for feature in feature_cols:
        if feature in df.columns:
            corr = df[feature].corr(df[target])
            correlations.append({
                'Feature': FEATURE_NAMES.get(feature, feature),
                'Correlation': corr,
                'Abs_Correlation': abs(corr),
                'Direction': '↑ Positive' if corr > 0 else '↓ Negative'
            })
    
    df_corr = pd.DataFrame(correlations).sort_values('Abs_Correlation', ascending=False)
    
    print("\n🏆 Top 10 Correlations:")
    for idx, row in df_corr.head(10).iterrows():
        print(f"  {idx+1}. {row['Feature']} {row['Direction']}: r={row['Correlation']:.3f}")
    
    return df_corr

In [ ]:
def plot_feature_importance(df_importance, target, method):
    """Plot feature importance bar chart."""
    plt.figure(figsize=(10, 8))
    
    df_plot = df_importance.head(15).copy()
    
    if 'Importance' in df_plot.columns:
        df_plot = df_plot.sort_values('Importance')
        y_col = 'Importance'
    else:
        df_plot = df_plot.sort_values('Abs_Correlation')
        y_col = 'Abs_Correlation'
    
    colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(df_plot)))
    plt.barh(df_plot['Feature'], df_plot[y_col], color=colors)
    
    plt.xlabel('Importance', fontsize=12, fontweight='bold')
    plt.ylabel('Feature', fontsize=12, fontweight='bold')
    plt.title(f'Feature Importance: {target.replace("_", " ").title()}\n({method})',
              fontsize=14, fontweight='bold', pad=20)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_comparison(results, target):
    """Compare all 3 methods side by side."""
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, (method, df_result) in enumerate(results.items()):
        ax = axes[idx]
        df_plot = df_result.head(10).copy()
        
        y_col = 'Importance' if 'Importance' in df_plot.columns else 'Abs_Correlation'
        df_plot = df_plot.sort_values(y_col)
        
        colors = plt.cm.plasma(np.linspace(0.3, 0.9, len(df_plot)))
        ax.barh(df_plot['Feature'], df_plot[y_col], color=colors)
        ax.set_xlabel('Importance', fontweight='bold')
        ax.set_title(method, fontsize=12, fontweight='bold')
        ax.grid(axis='x', alpha=0.3)
    
    fig.suptitle(f'Method Comparison: {target.replace("_", " ").title()}',
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
def analyze_target(df, target):
    """Run complete analysis for one target."""
    print(f"\n{'='*80}")
    print(f"ANALYZING: {target.replace('_', ' ').upper()}")
    print(f"{'='*80}")
    
    X, y, feature_names = prepare_data(df, target)
    if X is None:
        return None
    
    results = {}
    
    # Method 1: Random Forest
    df_rf = random_forest_importance(X, y, feature_names)
    results['Random Forest'] = df_rf
    plot_feature_importance(df_rf, target, 'Random Forest')
    
    # Method 2: Permutation
    df_perm = permutation_importance_analysis(X, y, feature_names)
    results['Permutation'] = df_perm
    plot_feature_importance(df_perm, target, 'Permutation Importance')
    
    # Method 3: Correlation
    df_clean = df[[f for f in FEATURE_COLS if f in df.columns] + [target]].dropna(subset=[target])
    if 'work_pressure' in df_clean.columns:
        df_clean['work_pressure'] = df_clean['work_pressure'].map({'low': 0, 'medium': 1, 'high': 2})
    df_corr = correlation_analysis(df_clean, target, feature_names)
    results['Correlation'] = df_corr
    
    # Comparison plot
    plot_comparison(results, target)
    
    # Consensus ranking
    print(f"\n{'='*80}")
    print(f"CONSENSUS RANKING")
    print(f"{'='*80}")
    
    all_features = set()
    for method_df in results.values():
        all_features.update(method_df['Feature'].tolist())
    
    consensus = []
    for feature in all_features:
        ranks = []
        for method_df in results.values():
            if feature in method_df['Feature'].values:
                rank = method_df[method_df['Feature'] == feature].index[0] + 1
            else:
                rank = 999
            ranks.append(rank)
        consensus.append({'Feature': feature, 'Avg_Rank': np.mean(ranks)})
    
    df_consensus = pd.DataFrame(consensus).sort_values('Avg_Rank')
    
    print("\n🏆 TOP 10 FEATURES (consensus across all methods):")
    for idx, row in df_consensus.head(10).iterrows():
        print(f"  {idx+1}. {row['Feature']} (Avg Rank: {row['Avg_Rank']:.1f})")
    
    return results, df_consensus

## 🚀 Run Analysis for All Targets

This will analyze each mental health outcome (stress, mood, energy, etc.) and show which behavioral features are most predictive.

In [ ]:
# Run analysis for all targets
all_results = {}
all_consensus = {}

for target in TARGET_COLS:
    if target in df.columns:
        results, consensus = analyze_target(df, target)
        all_results[target] = results
        all_consensus[target] = consensus
    else:
        print(f"\n⚠️  Skipping {target} - not in dataset")

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)
print(f"✓ Analyzed {len(all_results)} targets")
print(f"✓ Generated {len(all_results) * 4} visualizations")

## 📊 Summary: Top Predictive Features Across All Targets

View the consensus features that matter most for mental health outcomes.

In [ ]:
# Create summary DataFrame
summary_data = []
for target, consensus_df in all_consensus.items():
    top_5 = consensus_df.head(5)
    for idx, row in top_5.iterrows():
        summary_data.append({
            'Target': target.replace('_', ' ').title(),
            'Rank': idx + 1,
            'Feature': row['Feature'],
            'Avg_Rank': row['Avg_Rank']
        })

df_summary = pd.DataFrame(summary_data)

# Pivot to show top features per target
pivot = df_summary.pivot_table(
    index='Feature',
    columns='Target',
    values='Rank',
    aggfunc='first'
).fillna('-')

print("="*80)
print("TOP 5 FEATURES PER TARGET")
print("="*80)
display(pivot)

# Find universal predictors (top 5 for multiple targets)
feature_counts = df_summary['Feature'].value_counts()
universal = feature_counts[feature_counts >= 3]

if len(universal) > 0:
    print("\n🌟 UNIVERSAL PREDICTORS (important for 3+ targets):")
    for feature, count in universal.items():
        print(f"  • {feature}: Important for {count} targets")